# Hourly dataset for the project

In [32]:
import sys
print(sys.executable)

c:\ProgramData\anaconda3\python.exe


In [33]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_nord = qh[qh['MACROZONA'] == 'NORD']


qh_nord.tail()

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,MACROZONA,SBIL_MWH
ORAINI,,
2024-12-02 12:30:00,NORD,162.759
2024-12-02 12:45:00,NORD,192.662
2024-12-02 13:00:00,NORD,329.013
2024-12-02 13:15:00,NORD,271.264
2024-12-02 13:30:00,NORD,236.282


In [34]:
# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour
h_nord = qh_nord.resample('H').sum()

# Clean the 'MACROZONA' column to retain only 'NORD'
h_nord['MACROZONA'] = h_nord['MACROZONA'].str[:4]

h_nord = h_nord.drop(columns = ['MACROZONA'])
# Display the first few rows of the resampled data
h_nord.tail()

C:\Users\samueledelia\AppData\Local\Temp\3\ipykernel_10056\4018051853.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  h_nord = qh_nord.resample('H').sum()


,SBIL_MWH
ORAINI,
2024-12-02 09:00:00,-587.591
2024-12-02 10:00:00,209.173
2024-12-02 11:00:00,878.177
2024-12-02 12:00:00,541.433
2024-12-02 13:00:00,836.559


In [35]:
lags = [-1, -2, -3, -24]

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(h_nord['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged = df_sbil_lagged.resample('H').sum()

df_sbil_lagged

C:\Users\samueledelia\AppData\Local\Temp\3\ipykernel_10056\1260965639.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_sbil_lagged = df_sbil_lagged.resample('H').sum()


,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24
ORAINI,,,,
2019-01-01 00:00:00,0.000,0.000,0.000,0.000
2019-01-01 01:00:00,-111.900,0.000,0.000,0.000
2019-01-01 02:00:00,-112.451,-111.900,0.000,0.000
2019-01-01 03:00:00,-88.849,-112.451,-111.900,0.000
2019-01-01 04:00:00,127.792,-88.849,-112.451,0.000
...,...,...,...,...
2024-12-02 09:00:00,-1241.300,-1391.967,-737.221,-832.654
2024-12-02 10:00:00,-587.591,-1241.300,-1391.967,-241.100
2024-12-02 11:00:00,209.173,-587.591,-1241.300,-391.456


In [36]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,1843.786,0.0,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,1845.050,0.0,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,1845.647,0.0,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,983.111,0.0,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,978.775,0.0,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-03 19:00:00,42471.242,43112.042,3582.388,7727.555,23588.285,1119.317,2370.901,2515.126,0.0,None,...,NaN,0.0,2901.217,0.0,0.0,0.0,1825.0,41286.814,42612.042,2024-12-02 13:10:33.300
2024-12-03 20:00:00,40146.342,40250.790,3299.207,7400.575,21972.045,1108.544,2273.948,2418.206,0.0,None,...,NaN,0.0,2903.000,416.9,0.0,173.0,2010.0,38067.749,40146.342,2024-12-02 13:10:33.300
2024-12-03 21:00:00,37128.410,37138.993,3068.904,6767.221,20385.873,1024.545,2156.181,2217.647,0.0,None,...,NaN,0.0,2984.793,289.5,0.0,179.0,2081.0,34878.995,37128.410,2024-12-02 13:10:33.300


In [37]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mgp_volumes_nord.rename(columns={"NORD_PURCHASES": "MGP_NORD_PURCHASES", "NORD_SALES": "MGP_NORD_SALES"}, inplace=True)
mgp_volumes_nord = mgp_volumes_nord[~mgp_volumes_nord.index.duplicated(keep='first')]


#mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
#mi1_volumes = mi1_volumes.drop(columns=['MARKET'])
#mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
#mi1_volumes_nord.rename(columns={"NORD_PURCHASES": "MI1_NORD_PURCHASES", "NORD_SALES": "MI1_NORD_SALES"}, inplace=True)
#mi1_volumes_nord = mi1_volumes_nord[~mi1_volumes_nord.index.duplicated(keep='first')]

In [38]:
#mi1_volumes_nord.tail(40)

In [39]:
# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16_
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = """
SELECT * FROM POWER_UNBALANCE
"""
power_curve = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power_curve = power_curve.sort_values(by='TIMESTAMP')

power_curve = power_curve.pivot(index='TIMESTAMP', columns='SOURCE_ZONE', values='UNBALANCE_kW')

# Rename columns for clarity (optional, if needed)
power_curve.columns = [f"UNBALANCE_{col}" for col in power_curve.columns]

# Display the resulting DataFrame
power_curve = power_curve.resample('H').sum()

power_curve = power_curve[["UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD", "UNBALANCE_IDRO-PROGRAMMABILE_NORD", "UNBALANCE_SOLARE_NORD"]]

power_curve = power_curve.rename_axis("ORAINI")

power_curve

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\3\ipykernel_10056\3830827227.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  power_curve = power_curve.resample('H').sum()


,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,
2023-11-01 01:00:00,0.00,12447.43,0.000
2023-11-01 02:00:00,0.00,12130.31,0.000
2023-11-01 03:00:00,0.00,12297.07,0.000
2023-11-01 04:00:00,0.00,25895.64,0.000
2023-11-01 05:00:00,0.00,-28886.10,0.000
...,...,...,...
2024-12-02 10:00:00,136964.24,159797.68,226672.756
2024-12-02 11:00:00,107720.74,6219.02,145430.549
2024-12-02 12:00:00,-1879.02,-4503.58,73794.628


In [40]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM ENTSOE_DATA" 
entsoe_data = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
entsoe_data = entsoe_data.sort_values(by='ORAINI')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [41]:
entsoe_data = entsoe_data[entsoe_data['DOMAIN'] == 'LOAD']
entsoe_data = entsoe_data.drop(columns=['DOMAIN'])
# Convert ORAINI to datetime using the correct method
entsoe_data['ORAINI'] = pd.to_datetime(entsoe_data['ORAINI'], format='%Y%m%d%H%M')
entsoe_data.set_index('ORAINI', inplace=True)

day_ahead_load = entsoe_data[entsoe_data['SCOPE'] == 'DAY AHEAD']
day_ahead_load = day_ahead_load[['CODZONA', 'VALUE']]
day_ahead_load["FORECAST_TOTAL_LOAD_MW"] = day_ahead_load["VALUE"]
day_ahead_load = day_ahead_load.drop(columns="VALUE")

actual_load = entsoe_data[entsoe_data['SCOPE'] == 'ACTUAL']
actual_load = actual_load[['CODZONA', 'VALUE']]
actual_load["TOTAL_LOAD_MW"] = actual_load["VALUE"]
actual_load = actual_load.drop(columns="VALUE")

entsoe_load = pd.merge(day_ahead_load, actual_load, on=['ORAINI', 'CODZONA'])

subset_zonas = ['NORD']
mnord_load = entsoe_load[entsoe_load['CODZONA'].isin(subset_zonas)]
mnord_load = mnord_load.drop(columns=['CODZONA'])
mnord_load


,FORECAST_TOTAL_LOAD_MW,TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,11110.0,11511.0
2019-01-01 01:00:00,10675.0,11039.0
2019-01-01 02:00:00,10080.0,10430.0
2019-01-01 03:00:00,9793.0,9811.0
2019-01-01 04:00:00,9485.0,9474.0
...,...,...
2024-12-02 08:00:00,25147.0,25375.0
2024-12-02 09:00:00,25404.0,25308.0
2024-12-02 10:00:00,24968.0,24412.0


In [42]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [h_nord, df_sbil_lagged, mgp_volumes_nord, mnord_load, power_curve] #mi1_volumes_nord
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord_h_project = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

#df_nord_h = df_nord.drop(columns="MACROZONA")
df_nord_h_project = df_nord_h_project[df_nord_h_project.index >= '2024-08-27']
df_nord_h_project

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24,MGP_NORD_PURCHASES,MGP_NORD_SALES,FORECAST_TOTAL_LOAD_MW,TOTAL_LOAD_MW,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,,,,,,,,,,
2024-08-27 00:00:00,-6.856,-97.279,-51.994,-409.274,-270.368,16126.496,12677.375,17358.0,16029.0,-282467.38,0.0,0.0
2024-08-27 01:00:00,35.347,-6.856,-97.279,-51.994,-147.570,15370.564,11892.387,16570.0,15261.0,-287513.36,0.0,0.0
2024-08-27 02:00:00,-32.585,35.347,-6.856,-97.279,-214.758,14943.335,11202.301,16123.0,14842.0,-352965.19,0.0,0.0
2024-08-27 03:00:00,-117.998,-32.585,35.347,-6.856,21.760,14713.801,10895.785,15720.0,14628.0,-240815.93,0.0,0.0
2024-08-27 04:00:00,-297.574,-117.998,-32.585,35.347,-157.722,14760.303,11026.509,15615.0,14826.0,-255695.64,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-03 19:00:00,NaN,NaN,NaN,NaN,NaN,23588.285,22695.331,NaN,NaN,NaN,NaN,NaN
2024-12-03 20:00:00,NaN,NaN,NaN,NaN,NaN,21972.045,20290.151,NaN,NaN,NaN,NaN,NaN
2024-12-03 21:00:00,NaN,NaN,NaN,NaN,NaN,20385.873,18055.149,NaN,NaN,NaN,NaN,NaN


In [43]:
df_nord_h_project.tail(48)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24,MGP_NORD_PURCHASES,MGP_NORD_SALES,FORECAST_TOTAL_LOAD_MW,TOTAL_LOAD_MW,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,,,,,,,,,,
2024-12-02 00:00:00,-635.701,-513.084,-495.682,-505.760,-497.628,13329.175,7322.248,12855.0,12698.0,51556.41,-35675.34,5719.800
2024-12-02 01:00:00,-438.182,-635.701,-513.084,-495.682,-738.338,12889.051,6856.205,12313.0,12128.0,28001.87,107202.95,5719.800
2024-12-02 02:00:00,-663.813,-438.182,-635.701,-513.084,-437.529,12856.177,6706.560,12120.0,11967.0,31674.50,5830.25,5719.800
2024-12-02 03:00:00,-611.268,-663.813,-438.182,-635.701,-494.192,12996.550,6979.225,12088.0,12009.0,79385.16,6581.17,5719.800
2024-12-02 04:00:00,-735.972,-611.268,-663.813,-438.182,-469.891,13178.200,7483.776,12638.0,12624.0,26216.71,7515.29,5719.800
2024-12-02 05:00:00,-498.923,-735.972,-611.268,-663.813,-546.794,14495.689,8820.063,14268.0,14244.0,-1182.07,8908.35,5719.800
2024-12-02 06:00:00,-737.221,-498.923,-735.972,-611.268,-530.894,18378.332,12742.445,18468.0,18344.0,-26628.22,-47957.34,6148.950
2024-12-02 07:00:00,-1391.967,-737.221,-498.923,-735.972,-662.888,22010.780,17788.308,22753.0,22841.0,414267.54,-49991.33,-1410.535
2024-12-02 08:00:00,-1241.300,-1391.967,-737.221,-498.923,-686.006,24429.392,19060.510,25147.0,25375.0,622839.34,20719.73,-45439.153


In [44]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord_h_project.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord_h_project.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\3\ipykernel_10056\2037195458.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord_h_project.csv')
